### Defining costs

In [16]:
kulud = {
  "Kompleksteenus(€/tm)": 15,
  "Transport (€/tm)": 6,
  "Alghinna(%)": 10
} 

### Reading in EXCEL file for wood prices.

In [17]:
import pandas as pd

# Reading in EXCEL file for wood prices.
wood_prices = pd.read_excel('Data_Sources/Hinnakiri.xlsx')

print("Hinnakiri:")
print(wood_prices)

Hinnakiri:
                    Sortiment  Hind (€/tm)
0                     Ma palk           95
1                     Ku palk           95
2                Ks palk/pakk          110
3                     Hb palk           58
4                     Lm palk           60
5                     Lv palk           50
6                     Sa palk           40
7                     Ta palk           40
8                     Lh palk           40
9                     Sd palk           40
10  Lehtpuu palk (Va, Ja, Kp)           35
11                Ma peenpalk           64
12                Ku peenpalk           64
13              Ma paberipuit           62
14              Ku paberipuit           62
15              Ks paberipuit           50
16              Hb paberipuit           50
17                  Küttepuit           39
18                    Jäätmed            3


In [18]:
import requests
import json
from urllib.parse import quote

# Authorization API endpoint
auth_url = "https://lindaforest.collectivecrunch.net/api/v1/auth"

# Authorization headers with API keys
headers = {
    'linda-universal-api-key-id': '94faf919-235d-4ab5-a38a-8a7429825cfa',
    'linda-universal-api-key-secret': 'b90809e22f14d6743b911574160b3eb6'
}

try:
    # Send POST request with authorization headers
    auth_response = requests.post(auth_url, headers=headers, timeout=30)
    auth_response.raise_for_status()  # Gives out error if authorization fails
    auth_token = auth_response.json()['accessToken']
    print("Authorization successful.")

    # ---- SIMPLE VARIABLES (change here) ----
    country = "ee"                   # 'ee' or 'lv'
    property_id = "33801:001:1133"   # katastritunnus (':' kodeeritakse automaatselt)

    include_stands = True            # include stands
    include_predictions = True      # include predictions
    include_geometries = False       # include geometries (EPSG:4326)
    # ---------------------------------------

    # Build endpoint and query parameters
    base = "https://lindaforest.collectivecrunch.net/api/v1"
    property_encoded = quote(property_id, safe="")
    endpoint = f"{base}/scout/{country}/property/{property_encoded}"

    # API is waiting for 'true'/'false' string
    params = {
        "include_stands": str(include_stands).lower(),
        "include_predictions": str(include_predictions).lower(),
        "include_geometries": str(include_geometries).lower(),
    }

    # To get the data use Bearer token
    data_headers = {
        'Authorization': f'Bearer {auth_token}',
        'Accept': 'application/json'
    }

    # Get the data
    data_response = requests.get(endpoint, headers=data_headers, params=params, timeout=60)
    print("Request URL:", data_response.url)  # See the exact URL with parameters
    data_response.raise_for_status()  # Gives out error if data request fails
    data = data_response.json()
    print("Data successfully retrieved:", json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
    if err.response is not None:
        print(f"Response content: {err.response.text}")
except requests.exceptions.RequestException as err:
    print(f"Unexpected error: {err}")

Authorization successful.
Request URL: https://lindaforest.collectivecrunch.net/api/v1/scout/ee/property/33801%3A001%3A1133?include_stands=true&include_predictions=true&include_geometries=false
Data successfully retrieved: {
  "id": "33801:001:1133",
  "cadastral_id": "33801:001:1133",
  "total_area_ha": 6.64,
  "forest_area_ha": 6.6,
  "total_volume_m3": 1643,
  "cells_total_volume_m3": 1708,
  "pine_total_volume_m3": 170,
  "spruce_total_volume_m3": 963,
  "birch_total_volume_m3": 522,
  "other_deciduous_total_volume_m3": 53,
  "timber_volume_m3": 511,
  "pine_timber_volume_m3": 49,
  "spruce_timber_volume_m3": 302,
  "birch_timber_volume_m3": 146,
  "other_deciduous_timber_volume_m3": 14,
  "fiber_volume_m3": 923,
  "pine_fiber_volume_m3": 94,
  "spruce_fiber_volume_m3": 515,
  "birch_fiber_volume_m3": 287,
  "other_deciduous_fiber_volume_m3": 27,
  "stems": 4139,
  "pine_stems": 303,
  "spruce_stems": 1392,
  "birch_stems": 1634,
  "other_deciduous_stems": 809,
  "extra_volume": 65

### Reading in the API file for a specific cadastral.

In [19]:
import json

# Reading in JSON file for a specific cadastral.
e = data['stands']

# Create an empty list to store the new, transformed data records.
transformed_data_list = []
# Loop through each individual record (which is a dictionary) in the list.
for record in e:
    # Extract the stand number for the current record.
    stand_number = record["stand_number"]
    # Now you can safely access the values using keys from this single record.
    # Create a list of dictionaries for each species in the current record.
    species_data = [
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Pine",
            "Kõrgus m": record["pine_bam_height_m"],
            "Diameeter cm": record["pine_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["pine_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["pine_total_volume_m3_ha"] or 0)
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Spruce",
            "Kõrgus m": record["spruce_bam_height_m"],
            "Diameeter cm": record["spruce_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["spruce_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["spruce_total_volume_m3_ha"] or 0)
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Birch",
            "Kõrgus m": record["birch_bam_height_m"],
            "Diameeter cm": record["birch_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["birch_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["birch_total_volume_m3_ha"] or 0)
        
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Other Deciduous",
            "Kõrgus m": record["other_deciduous_bam_height_m"],
            "Diameeter cm": record["other_deciduous_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["other_deciduous_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["other_deciduous_total_volume_m3_ha"] or 0),
            
        }
    ]
    # Add the newly created species data to our main list.
    transformed_data_list.extend(species_data)
# Finally, create a single DataFrame from the combined list of all records.
API_cadastral_data = pd.DataFrame(transformed_data_list)
print(API_cadastral_data)

   Eraldise nr          Puuliik  Kõrgus m  Diameeter cm  Pindala ha  \
0            0             Pine      14.8            23        1.04   
1            0           Spruce      18.0            23        1.04   
2            0            Birch      15.1            17        1.04   
3            0  Other Deciduous       8.6            11        1.04   
4            1             Pine      22.8            30        1.42   
5            1           Spruce      23.0            28        1.42   
6            1            Birch      22.0            22        1.42   
7            1  Other Deciduous       0.0             0        1.42   
8            2             Pine      23.2            33        1.94   
9            2           Spruce      23.0            29        1.94   
10           2            Birch      22.7            22        1.94   
11           2  Other Deciduous      23.3            28        1.94   
12           3             Pine      24.4            32        1.65   
13    

### Reading in EXCEL file for tree names.

In [20]:
# Reading in EXCEL file for tree names.
tree_name = pd.read_excel('Data_Sources/Puu_nimetused_EE_ENG.xlsx')

print("Puude nimetused EE/ENG:")
print(tree_name.head())


Puude nimetused EE/ENG:
  Name_EE         Name_ENG
0      MA             Pine
1      KU           Spruce
2      KS            Birch
3      LV  Other Deciduous
4      HB              NaN


### Reading in EXCEL file for relative heights.

In [21]:
# Reading in EXCEL file for relative height.
relative_heights = pd.read_excel('Data_Sources/Suhtelised_tugikõrgused.xlsx')

print("Suhtelised tugikõrgused:")
print(relative_heights)


Suhtelised tugikõrgused:
     d      MA     KU      KS      LV      HB      LM      TA      SA      VA  \
0    8  0.4300  0.350  0.5500  0.5500  0.5200  0.5200  0.3800  0.3800  0.3800   
1    9  0.5000  0.410  0.6000  0.6000  0.5700  0.5700  0.4400  0.4400  0.4400   
2   10  0.5500  0.470  0.6400  0.6400  0.6200  0.6200  0.5000  0.5000  0.5000   
3   11  0.6100  0.530  0.6800  0.6800  0.6600  0.6600  0.5500  0.5500  0.5500   
4   12  0.6500  0.580  0.7200  0.7200  0.7000  0.7000  0.6000  0.6000  0.6000   
5   13  0.7000  0.630  0.7600  0.7600  0.7400  0.7400  0.6500  0.6500  0.6500   
6   14  0.7400  0.680  0.7900  0.7900  0.7700  0.7700  0.7000  0.7000  0.7000   
7   15  0.7700  0.720  0.8200  0.8200  0.8000  0.8000  0.7400  0.7400  0.7400   
8   16  0.8100  0.760  0.8400  0.8400  0.8300  0.8300  0.7700  0.7700  0.7700   
9   17  0.8400  0.800  0.8700  0.8700  0.8600  0.8600  0.8100  0.8100  0.8100   
10  18  0.8700  0.830  0.8900  0.8900  0.8800  0.8800  0.8400  0.8400  0.8400   
11 

### Reading in EXCEL file for log volume distribution

In [22]:
# Reading in EXCEL file for type proportions.
log_volume_distribution = pd.read_excel('Data_Sources/Mahutabel.xlsx')

print("Mahutabel:")
print(log_volume_distribution)

Mahutabel:
     d klass+pl+h24 x m  kõrgus      palk     peenp     paber      küte  \
0                 8MA16     8.0  0.000000  0.000000  0.000000  0.500000   
1                12MA16    11.0  0.000000  0.000000  0.714286  0.000000   
2                16MA16    13.0  0.000000  0.333333  0.400000  0.000000   
3                20MA16    15.0  0.000000  0.680000  0.080000  0.000000   
4                24MA16    16.0  0.289474  0.368421  0.105263  0.000000   
...                 ...     ...       ...       ...       ...       ...   
3205             36PN27    32.0  0.000000  0.000000  0.000000  0.794521   
3206             40PN27    33.0  0.000000  0.000000  0.000000  0.795699   
3207             44PN27    34.0  0.000000  0.000000  0.000000  0.800000   
3208             48PN27    35.0  0.000000  0.000000  0.000000  0.806452   
3209             52PN27    35.0  0.000000  0.000000  0.000000  0.804805   

       jäätmed  kokku  
0     0.500000    1.0  
1     0.285714    1.0  
2     0.266667  

### Merging dataframes API_cadastral_data and tree_name.

In [23]:
# Merging two dataframes on main species names and adding Name_EE, Name_ENG.
cadastral_data = pd.merge(API_cadastral_data, tree_name, left_on='Puuliik', right_on='Name_ENG', how='left')

### Adding a 'Relative height' and 'h24' column to the table.

In [24]:
import numpy as np

cadastral_data['Suhteline tugikõrgus'] = np.nan

for index, row in cadastral_data.iterrows():
    name_ee = row['Name_EE']
    diameter_cm = row['Diameeter cm']
    try:
        # Look up the correct value from relative_heights table.
        h24_value = relative_heights.loc[relative_heights['d'] == diameter_cm, name_ee]
        # When the value is found add it to the column 
        if not h24_value.empty:
            cadastral_data.at[index, 'Suhteline tugikõrgus'] = h24_value.values[0]
    except KeyError:
        # When there isn´t a value add NaN to the row
        pass

# Adding a 'h24' column to the table.
cadastral_data['h24'] = cadastral_data.apply(
    lambda row: 16 if (pd.notna(row['Kõrgus m']) and pd.notna(row['Suhteline tugikõrgus']) and row['Suhteline tugikõrgus'] != 0 and (np.ceil(row['Kõrgus m'] / row['Suhteline tugikõrgus'])) < 16) else (int(np.ceil(row['Kõrgus m'] / row['Suhteline tugikõrgus'])) if pd.notna(row['Kõrgus m']) and pd.notna(row['Suhteline tugikõrgus']) and row['Suhteline tugikõrgus'] != 0 else np.nan),
    axis=1)
#
cadastral_data['h24'] = cadastral_data['h24'].astype('Int64')

### Adding a 'Diameetri klass' column to the table

In [25]:
# Adding a 'Diameetri klass' column to the table.
def diameter_category(diameter_cm):
    if 5 <= diameter_cm <= 52:
        # Round up to the nearest multiple of 4 (aligned with 8, 12, 16, …, 52)
        return ((diameter_cm + 3) // 4) * 4
    elif diameter_cm > 52:
        return 52
    return None
cadastral_data['Diameetri klass'] = (
    cadastral_data['Diameeter cm']
    .apply(diameter_category)# Using .apply() to use the function on 'Diameeter cm' values.
    .astype('Int64')# Getting 'Diameetri klass' column to Int64 type.
)

### Adding a 'Sortimendi jaotusklass' column to the table

In [26]:
cadastral_data['Sortimendi jaotusklass'] = cadastral_data['Diameetri klass'].astype(str) + '' + cadastral_data['Name_EE'].astype(str) + '' + cadastral_data['h24'].astype(str)

### Merging log_volume_distribution with cadastral_data on 'Sortimendi jaotusklass' to get the Mahu jaotus.

In [27]:
cadastral_data = pd.merge(cadastral_data, 
                     log_volume_distribution, 
                     left_on='Sortimendi jaotusklass', 
                     right_on='d klass+pl+h24 x m', 
                     how='inner')

# Calculating proportions for each row
columns_to_multiply = ['palk', 'peenp', 'paber', 'küte', 'jäätmed']

# A loop that multiplys type proportion columns by 'Tagavara m3' column values.
for column in columns_to_multiply:
    cadastral_data[column] = cadastral_data[column] * cadastral_data['Tagavara m3']

# Dropping unnecessary columns from the table.
cadastral_data = cadastral_data.drop(columns=['d klass+pl+h24 x m', 'kõrgus', 'kokku','Name_ENG'])

print("Lõplik katastriandmete tabel:")
print(cadastral_data)

Lõplik katastriandmete tabel:
   Eraldise nr          Puuliik  Kõrgus m  Diameeter cm  Pindala ha  \
0            0             Pine      14.8            23        1.04   
1            0           Spruce      18.0            23        1.04   
2            0            Birch      15.1            17        1.04   
3            0  Other Deciduous       8.6            11        1.04   
4            1             Pine      22.8            30        1.42   
5            1           Spruce      23.0            28        1.42   
6            1            Birch      22.0            22        1.42   
7            2             Pine      23.2            33        1.94   
8            2           Spruce      23.0            29        1.94   
9            2            Birch      22.7            22        1.94   
10           2  Other Deciduous      23.3            28        1.94   
11           3             Pine      24.4            32        1.65   
12           3           Spruce      24.9      

### Creating a new dataframe to sum up 'Maht' and 'Hind'

In [28]:
# Define the mapping between Sortiment, source (Name_EE + column), and price names
mappings = [
    ("Ma palk",        ("MA", "palk"),   "Ma palk"),
    ("Ku palk",        ("KU", "palk"),   "Ku palk"),
    ("Ks palk/pakk",   ("KS", "palk"),   "Ks palk/pakk"),
    ("Teised liigid/Lv palk", ("LV", "palk"), "Lv palk"),
    ("Ma peenpalk",    ("MA", "peenp"),  "Ma peenpalk"),
    ("Ku peenpalk",    ("KU", "peenp"),  "Ku peenpalk"),
    ("Ma paberipuit",  ("MA", "paber"),  "Ma paberipuit"),
    ("Ku paberipuit",  ("KU", "paber"),  "Ku paberipuit"),
    ("Ks paberipuit",  ("KS", "paber"),  "Ks paberipuit"),
    ("Küttepuit",      (None, "küte"),   "Küttepuit"),
    ("Jäätmed",        (None, "jäätmed"),"Jäätmed"),
]
rows = []
for sortiment, (name_ee, col), price_name in mappings:
    if name_ee:  # sum by Name_EE + column
        volume = cadastral_data.loc[cadastral_data["Name_EE"] == name_ee, col].sum()
    else:        # global sum
        volume = cadastral_data[col].sum()
    # find matching price
    price = wood_prices.loc[wood_prices["Sortiment"] == price_name, "Hind (€/tm)"].squeeze()
    rows.append({
        "Sortiment": sortiment,
        "Maht (tm)": volume,
        "Hind (€)": volume * price
    })
# Create DataFrame
maht_hind_kokku = pd.DataFrame(rows)
# Add total row
maht_hind_kokku.loc[len(maht_hind_kokku)] = {
    "Sortiment": "Kokku",
    "Maht (tm)": maht_hind_kokku["Maht (tm)"].sum(),
    "Hind (€)": maht_hind_kokku["Hind (€)"].sum()
}
# Round results
maht_hind_kokku["Maht (tm)"] = maht_hind_kokku["Maht (tm)"].round(1)
maht_hind_kokku["Hind (€)"] = maht_hind_kokku["Hind (€)"].round(1)

print("Lõplik mahu ja hinna tabel:")
print(maht_hind_kokku)

Lõplik mahu ja hinna tabel:
                Sortiment  Maht (tm)  Hind (€)
0                 Ma palk      113.4   10772.9
1                 Ku palk      622.1   59103.4
2            Ks palk/pakk      193.6   21296.7
3   Teised liigid/Lv palk       25.1    1254.6
4             Ma peenpalk       15.4     985.8
5             Ku peenpalk      119.0    7614.8
6           Ma paberipuit        8.8     544.4
7           Ku paberipuit       38.1    2363.0
8           Ks paberipuit      201.1   10057.5
9               Küttepuit       31.0    1207.6
10                Jäätmed      342.1    1026.4
11                  Kokku     1709.8  116227.2


### Creating a new dataframe to get recommended starting bid

In [29]:
# Extract key values
maht_kokku = maht_hind_kokku.at[maht_hind_kokku.index[maht_hind_kokku['Sortiment'] == 'Kokku'][0], 'Maht (tm)']
hind_kokku = maht_hind_kokku.at[maht_hind_kokku.index[maht_hind_kokku['Sortiment'] == 'Kokku'][0], 'Hind (€)']
maht_jaatmeteta = maht_hind_kokku.loc[
    ~maht_hind_kokku['Sortiment'].isin(['Jäätmed', 'Kokku']),
    'Maht (tm)'
].sum()
# Calculations
kulud_tm = kulud["Kompleksteenus(€/tm)"] + kulud["Transport (€/tm)"]
kulud_jaatmeteta = maht_jaatmeteta * kulud_tm
tulud_kulud_jaatmeteta = hind_kokku - kulud_jaatmeteta
soovituslik_alghind = tulud_kulud_jaatmeteta * (1 - kulud["Alghinna(%)"] / 100)
# Build result table
results = {
    "Maht kokku":              (maht_kokku, "tm"),
    "Tulud kokku":              (hind_kokku, "€"),
    "kulud (jäätmeteta)":      (kulud_jaatmeteta, "€"),
    "Tulud-kulud (jäätmeteta)":(tulud_kulud_jaatmeteta, "€"),
    "Soovituslik alghind":     (soovituslik_alghind, "€"),
}
cadastral_data_summary = (
    pd.DataFrame.from_dict(results, orient="index", columns=["Väärtus", "Ühik"])
    .assign(Väärtus=lambda df: df["Väärtus"].round(1))
)

print("Katastriüksuse hinnanguline väärtus:")
print(cadastral_data_summary)

Katastriüksuse hinnanguline väärtus:
                           Väärtus Ühik
Maht kokku                  1709.8   tm
Tulud kokku               116227.2    €
kulud (jäätmeteta)         28719.6    €
Tulud-kulud (jäätmeteta)   87507.6    €
Soovituslik alghind        78756.8    €


### Adding a interactive browser control.

In [30]:
import io
import json
import requests
from urllib.parse import quote

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Abifunktsioon väljade loomiseks
def bf(desc, value=0.0, minv=0.0, width='340px'):
    return widgets.BoundedFloatText(
        value=value, min=minv, description=desc,
        style={'description_width':'initial'},
        layout=widgets.Layout(width=width)
    )

# Vidinad: kulud
show_details     = widgets.Checkbox(value=False, description="Näita detailseid tabeleid (hinnakiri, sortimendid)", indent=False)
komplekt_widget  = bf('Kompleksteenus(€/tm):', 15)
transport_widget = bf('Transport (€/tm):', 6)
alghind_widget   = bf('Alghinna(%):', 10)

# Vidinad: hinnad (kui Excelit ei laeta)
price_names = [
    "Ma palk","Ku palk","Ks palk/pakk","Lv palk",
    "Ma peenpalk","Ku peenpalk",
    "Ma paberipuit","Ku paberipuit","Ks paberipuit","Hb paberipuit",
    "Küttepuit","Jäätmed"
]
wood_price_widgets = {name: bf(f"{name}:", 0) for name in price_names}
prices_grid = widgets.GridBox(
    children=list(wood_price_widgets.values()),
    layout=widgets.Layout(grid_template_columns='repeat(3, 350px)', grid_gap='6px')
)

# Vidinad: hinnakirja Excel (valikuline)
upload_widget = widgets.FileUpload(accept='.xlsx', multiple=False)

# Vidinad: API sisendid
api_id_widget     = widgets.Text(description='API Key ID:', placeholder='sisesta', layout=widgets.Layout(width='420px'), style={'description_width':'initial'})
api_secret_widget = widgets.Password(description='API Key Secret:', placeholder='sisesta', layout=widgets.Layout(width='420px'), style={'description_width':'initial'})
country_widget    = widgets.Dropdown(options=['ee','lv'], value='ee', description='Country:', layout=widgets.Layout(width='220px'), style={'description_width':'initial'})
prop_widget       = widgets.Text(description='Property ID:', placeholder='nt 33801:001:1133', layout=widgets.Layout(width='300px'), style={'description_width':'initial'})
incl_stands       = widgets.Checkbox(value=True, description='include_stands', indent=False)
incl_preds        = widgets.Checkbox(value=True, description='include_predictions', indent=False)
incl_geoms        = widgets.Checkbox(value=False, description='include_geometries', indent=False)

calc_button = widgets.Button(description="Arvuta")
output      = widgets.Output()

def _error(msg):
    display(widgets.HTML(f"<span style='color:#b00020;font-weight:600'>{msg}</span>"))

def _get_uploaded_content(upl):
    v = upl.value
    if not v:
        return None
    item = next(iter(v.values())) if isinstance(v, dict) else (v[0] if isinstance(v, (list, tuple)) else v)
    content = item.get('content') if isinstance(item, dict) else getattr(item, 'content', None)
    return content.tobytes() if isinstance(content, memoryview) else content

def _auth_and_fetch_property(api_id, api_secret, country, property_id, include_stands=True, include_predictions=False, include_geometries=False):
    base = "https://lindaforest.collectivecrunch.net/api/v1"
    # Auth
    auth_resp = requests.post(
        f"{base}/auth",
        headers={
            "linda-universal-api-key-id": api_id.strip(),
            "linda-universal-api-key-secret": api_secret.strip(),
        },
        timeout=30
    )
    auth_resp.raise_for_status()
    token = auth_resp.json()["accessToken"]

    # Data
    endpoint = f"{base}/scout/{country}/property/{quote(property_id, safe='')}"
    params = {
        "include_stands": str(bool(include_stands)).lower(),
        "include_predictions": str(bool(include_predictions)).lower(),
        "include_geometries": str(bool(include_geometries)).lower(),
    }
    data_resp = requests.get(
        endpoint,
        headers={"Authorization": f"Bearer {token}", "Accept": "application/json"},
        params=params,
        timeout=60
    )
    return data_resp

# Tee _build_maht_hind_kokku_from_json võimeline vastu võtma ka dict payload'i
def _build_maht_hind_kokku_from_json(json_input):
    # json_input võib olla dict (API payload) või bytes/str
    if isinstance(json_input, dict):
        data = json_input
    else:
        try:
            if hasattr(json_input, "read"):
                raw = json_input.read()
            else:
                raw = json_input
            if isinstance(raw, memoryview):
                raw = raw.tobytes()
            if isinstance(raw, (bytes, bytearray)):
                text = raw.decode("utf-8", errors="replace")
            elif isinstance(raw, str):
                text = raw
            else:
                text = bytes(raw).decode("utf-8", errors="replace")
            data = json.loads(text)
        except Exception as e:
            _error(f"JSON lugemise viga: {e}")
            return None

    stands = data.get('stands')
    if not stands:
        _error("API vastuses puudub 'stands'. Veendu, et include_stands=True.")
        return None

    rows = []
    for r in stands:
        area = (r.get("total_area_ha") or 0)  # None->0
        def rec(name_eng, h_key, d_key, v_key):
            h = r.get(h_key)
            d = r.get(d_key)
            v_ha = r.get(v_key)
            if v_ha is None:
                return
            rows.append({
                "Eraldise nr": r.get("stand_number"),
                "Puuliik": name_eng,
                "Kõrgus m": h,
                "Diameeter cm": d,
                "Pindala ha": float(area) or 0.0,
                "Tihedus m3/ha": float(v_ha) if v_ha is not None else 0.0,
                "Tagavara m3": (float(area) or 0.0) * (float(v_ha) if v_ha is not None else 0.0)
            })
        rec("Pine", "pine_bam_height_m", "pine_bam_dbh_cm", "pine_total_volume_m3_ha")
        rec("Spruce", "spruce_bam_height_m", "spruce_bam_dbh_cm", "spruce_total_volume_m3_ha")
        rec("Birch", "birch_bam_height_m", "birch_bam_dbh_cm", "birch_total_volume_m3_ha")
        rec("Other Deciduous", "other_deciduous_bam_height_m", "other_deciduous_bam_dbh_cm", "other_deciduous_total_volume_m3_ha")

    if not rows:
        _error("API-st ei saadud ühtegi liikide rida.")
        return None

    df = pd.DataFrame(rows)

    # Nimekaardistus + arvutused (eeldab, et tree_name, relative_heights, log_volume_distribution on laaditud varasemates rakkudes)
    df = pd.merge(df, tree_name, left_on='Puuliik', right_on='Name_ENG', how='left')

    df['Suhteline tugikõrgus'] = np.nan
    for i, row in df.iterrows():
        name_ee = row['Name_EE']
        d = row['Diameeter cm']
        try:
            h24_val = relative_heights.loc[relative_heights['d'] == d, name_ee]
            if not h24_val.empty:
                df.at[i, 'Suhteline tugikõrgus'] = h24_val.values[0]
        except KeyError:
            pass

    def _calc_h24(r):
        if pd.notna(r['Kõrgus m']) and pd.notna(r['Suhteline tugikõrgus']) and r['Suhteline tugikõrgus'] != 0:
            val = int(np.ceil(r['Kõrgus m'] / r['Suhteline tugikõrgus']))
            return 16 if val < 16 else val
        return np.nan
    df['h24'] = df.apply(_calc_h24, axis=1).astype('Int64')

    def diameter_category(d):
        if pd.isna(d):
            return None
        if 5 <= d <= 52:
            return ((int(d) + 3) // 4) * 4
        if d > 52:
            return 52
        return None
    df['Diameetri klass'] = df['Diameeter cm'].apply(diameter_category).astype('Int64')

    df['Sortimendi jaotusklass'] = df['Diameetri klass'].astype(str) + df['Name_EE'].astype(str) + df['h24'].astype(str)

    merged = pd.merge(
        df,
        log_volume_distribution,
        left_on='Sortimendi jaotusklass',
        right_on='d klass+pl+h24 x m',
        how='inner'
    )

    for c in ['palk', 'peenp', 'paber', 'küte', 'jäätmed']:
        merged[c] = merged[c] * merged['Tagavara m3']

    merged = merged.drop(columns=['d klass+pl+h24 x m', 'kõrgus', 'kokku', 'Name_ENG'], errors='ignore')

    mappings = [
        ("Ma palk", ("MA","palk"), "Ma palk"),
        ("Ku palk", ("KU","palk"), "Ku palk"),
        ("Ks palk/pakk", ("KS","palk"), "Ks palk/pakk"),
        ("Teised liigid/Lv palk", ("LV","palk"), "Lv palk"),
        ("Ma peenpalk", ("MA","peenp"), "Ma peenpalk"),
        ("Ku peenpalk", ("KU","peenp"), "Ku peenpalk"),
        ("Ma paberipuit", ("MA","paber"), "Ma paberipuit"),
        ("Ku paberipuit", ("KU","paber"), "Ku paberipuit"),
        ("Ks paberipuit", ("KS","paber"), "Ks paberipuit"),
        ("Küttepuit", (None,"küte"), "Küttepuit"),
        ("Jäätmed", (None,"jäätmed"), "Jäätmed"),
    ]
    out_rows = []
    for sortiment, (name_ee, col), price_name in mappings:
        if name_ee:
            vol = merged.loc[merged["Name_EE"] == name_ee, col].sum()
        else:
            vol = merged[col].sum()
        out_rows.append({"Sortiment": sortiment, "Maht (tm)": vol, "Hind (€)": 0.0})
    mhk = pd.DataFrame(out_rows)
    mhk.loc[len(mhk)] = {"Sortiment": "Kokku", "Maht (tm)": mhk["Maht (tm)"].sum(), "Hind (€)": 0.0}
    mhk["Maht (tm)"] = mhk["Maht (tm)"].round(1)
    return mhk

def arvuta(_):
    with output:
        clear_output()

        # 1) Kulud
        kulud = {
            "Kompleksteenus(€/tm)": komplekt_widget.value,
            "Transport (€/tm)": transport_widget.value,
            "Alghinna(%)": alghind_widget.value
        }
        if any(v < 0 for v in kulud.values()):
            _error("Kulude väärtused ei tohi olla negatiivsed.")
            return

        # 2) Hinnad (Excel või manuaalne)
        xlsx_content = _get_uploaded_content(upload_widget)
        if xlsx_content is not None:
            wood_prices_in = pd.read_excel(io.BytesIO(xlsx_content))
            need_cols = {"Sortiment","Hind (€/tm)"}
            if not need_cols.issubset(wood_prices_in.columns):
                _error(f"Excel peab sisaldama veerge: {need_cols}")
                if show_details.value:
                    display(wood_prices_in.head())
                return
            wood_prices_in["Hind (€/tm)"] = pd.to_numeric(wood_prices_in["Hind (€/tm)"], errors="coerce")
            if wood_prices_in["Hind (€/tm)"].isna().any():
                _error("Veerus 'Hind (€/tm)' on mittearvulisi väärtusi.")
                if show_details.value:
                    display(wood_prices_in)
                return
            if (wood_prices_in["Hind (€/tm)"] < 0).any():
                _error("Hinnad ei tohi olla negatiivsed (Excel).")
                if show_details.value:
                    display(wood_prices_in[wood_prices_in["Hind (€/tm)"] < 0])
                return
            if show_details.value:
                display(wood_prices_in)
            prices = dict(wood_prices_in[["Sortiment","Hind (€/tm)"]].values)
        else:
            prices = {k: w.value for k, w in wood_price_widgets.items()}
            neg = {k:v for k,v in prices.items() if v < 0}
            if neg:
                _error("Hinnad ei tohi olla negatiivsed (manuaalne sisestus).")
                if show_details.value:
                    display(pd.DataFrame(list(neg.items()), columns=["Sortiment","Väärtus"]))
                return
            if show_details.value:
                display(pd.DataFrame(list(prices.items()), columns=["Sortiment","Hind (€/tm)"]))

        # 3) API päring (kasutaja sisestab võtmed ja katastri)
        api_id = api_id_widget.value.strip()
        api_secret = api_secret_widget.value.strip()
        country = country_widget.value
        prop_id = prop_widget.value.strip()
        if not api_id or not api_secret or not prop_id:
            _error("Täida API Key ID, API Key Secret ja Property ID.")
            return

        try:
            resp = _auth_and_fetch_property(
                api_id, api_secret, country, prop_id,
                include_stands=incl_stands.value,
                include_predictions=incl_preds.value,
                include_geometries=incl_geoms.value
            )
            if show_details.value:
                print("Request URL:", resp.url)
            resp.raise_for_status()
            payload = resp.json()
        except requests.exceptions.HTTPError as e:
            _error(f"API viga: {e}")
            try:
                print("Response:", resp.text[:1000])
            except Exception:
                pass
            return
        except Exception as e:
            _error(f"API viga: {e}")
            return

        # 4) Ehita mahtude tabel API payloadist
        mhk = _build_maht_hind_kokku_from_json(payload)
        if mhk is None:
            return

        # 5) Hinnad peale
        def _norm(s): return str(s).strip().lower()
        alias = {
            "teised liigid/lv palk": "lv palk",
            "teised liigid paberipuit": "hb paberipuit",
        }
        prices_norm = {_norm(k): float(v) for k,v in prices.items()}

        mask = mhk["Sortiment"] != "Kokku"
        keys = mhk.loc[mask,"Sortiment"].map(_norm).map(lambda k: alias.get(k,k))
        price_series = keys.map(prices_norm).fillna(0.0)
        mhk.loc[mask,"Hind (€)"] = mhk.loc[mask,"Maht (tm)"].values * price_series.values

        if "Kokku" in mhk["Sortiment"].values:
            idx = mhk.index[mhk["Sortiment"] == "Kokku"][0]
            mhk.at[idx, "Hind (€)"] = mhk.loc[mask, "Hind (€)"].sum()

        # 6) Summaarsed näitajad
        hind_kokku = float(mhk.loc[mask,"Hind (€)"].sum())
        maht_kokku = float(mhk.loc[mhk["Sortiment"]=="Kokku","Maht (tm)"].iloc[0])
        maht_jaatmeteta = float(mhk.loc[~mhk["Sortiment"].isin(["Jäätmed","Kokku"]),"Maht (tm)"].sum())

        kulud_tm = kulud["Kompleksteenus(€/tm)"] + kulud["Transport (€/tm)"]
        kulud_jaatmeteta = maht_jaatmeteta * kulud_tm
        tulud_kulud_jaatmeteta = hind_kokku - kulud_jaatmeteta
        soovituslik_alghind = tulud_kulud_jaatmeteta * (1 - kulud["Alghinna(%)"]/100)

        if show_details.value:
            display(mhk[["Sortiment","Maht (tm)","Hind (€)"]])

        # 7) Tulemused soovitud järjekorras ja nimetustega
        results = pd.DataFrame({
            "Väärtus":[maht_kokku, hind_kokku, kulud_jaatmeteta, tulud_kulud_jaatmeteta, soovituslik_alghind],
            "Ühik":["tm","€","€","€","€"]
        }, index=[
            "Maht kokku",
            "Hind kokku",
            "Kulud (jäätmeteta)",
            "Tulud-kulud (jäätmeteta)",
            "Soovituslik alghind"
        ])
        results["Väärtus"] = results["Väärtus"].round(1)
        display(results)

def _ensure_single_handler(btn, fn):
    for cb in list(getattr(btn._click_handlers, "callbacks", [])):
        btn.on_click(cb, remove=True)
    btn.on_click(fn)

_ensure_single_handler(calc_button, arvuta)

display(widgets.VBox([
    widgets.Label("Sisesta kulude väärtused:"),
    komplekt_widget, transport_widget, alghind_widget,
    widgets.Label("Sisesta puidu hinnad käsitsi või laadi üles Excel:"),
    prices_grid,
    widgets.Label("Või laadi üles hinnakiri (.xlsx) veergudega 'Sortiment' ja 'Hind (€/tm)':"),
    upload_widget,
    widgets.HTML("<hr/>"),
    widgets.Label("API sisendid:"),
    widgets.HBox([api_id_widget, api_secret_widget]),
    widgets.HBox([country_widget, prop_widget, widgets.HBox([incl_stands, incl_preds, incl_geoms])]),
    show_details,
    calc_button,
    output
]))